## Creating a program in the IR of the stam language

First, let's import the necessary imports.

In [560]:
from hw.cpu.assembler import asm
from sw.basics import asm_interp
from sw.compiler.backend import CompilerBackend
from sw.compiler.parser import IRParser
from sw.verify.stasm_to_chcs import create_chcs
from z3 import And, Ints, Int, Array, IntSort, ForAll, Implies, Or
import z3
import graphviz
z3.set_param(proof=True)
from sw.verify.stasm_to_chcs import create_chcs

p = IRParser()
cb = CompilerBackend()

Now let's write a function in StamlIR that draws a square. First, we'll add the functions given to us as examples

In [561]:
prog = '''
/*
main =
    let l = 11, h = 20
    horiz l (h + 1) l;
    vert h (l + 1) (h + 1);
    horiz (h - 1) l h;
    vert l h l
*/
main(0) =
    11; 20;
    ignore horiz $-1 ($-2 + 1) $-1;
    ignore vert  $-2 ($-1 + 1) ($-2 + 1);
    ignore horiz ($-2 - 1) $-1 $-2;
    ignore vert  $-1 $-2 $-1;
    0

memset_skip(4) =
    if 0 < $1 then mem_poke $0 $3; .memset_skip ($3 + $2) $2 ($1 - 1) $0 else 0

/*
# memor_skip start skip cnt value : unit =
#   if 0 < cnt then mem_poke (value | mem_peek(start)) start;
#                   memor_skip (start + skip) skip (cnt - 1) value
#              else ()
*/
memor_skip(4) =
    if 0 < $1 then mem_poke ($0 | mem_peek $3) $3; .memor_skip ($3 + $2) $2 ($1 - 1) $0 else 0

/*
# wait () : u16 =
#  let seq = mem_peek 0xc001
#  let aux () =
#    if (mem_peek 0xc001) == seq then aux ()
#    else (mem_peek 0xc000)
#  aux ()
*/
wait(0) = mem_peek 0xc001; wait_aux
wait_aux(0) =
    if (mem_peek 0xc001) - $1 /* < access closure */
    then mem_peek 0xc000 else .wait_aux

/*
# block x y =
#   let mask = (x & 1 ? 0xff00 : 0x00ff) in
#   memor_skip (0xa000 + 0x80 * y + (x >> 1)) 0x10 0x08 mask
*/
block(2) =
    (if $1 & 1 then 0xff00 else 0x00ff);
    memor_skip (0xa000 + 0x80 * $0 + ($1 >> 1)) 0x10 0x08 $-1
/*
# memor_shr_skip start skip cnt value shft : unit =
#   if 0 < cnt then mem_poke ((value << shft) | mem_peek(start)) start;
#                   memor_shr_skip (start + skip) skip (cnt - 1) (value >> 1)
#              else ()
*/
memor_shr_skip(5) =
    if 0 < $2 then ignore mem_poke (($1 << $0) | mem_peek $4) $4;
                   memor_shr_skip ($4 + $3) $3 ($2 - 1) ($1 >> 1) $0
              else 0

/*
# memor_shl_skip start skip cnt value shft : unit =
#   if 0 < cnt then mem_poke ((value >> shft) | mem_peek(start)) start;
#                   memor_shr_skip (start + skip) skip (cnt - 1) (value << 1)
#              else ()
*/
memor_shl_skip(5) =
    if 0 < $2 then ignore mem_poke (($1 >> $0) | mem_peek $4) $4;
                   memor_shl_skip ($4 + $3) $3 ($2 - 1) ($1 << 1) $0
              else 0

/* horiz x0 x1 y  */
horiz(3) =
    (if $2 < $1 then 1 else (if $1 < $2 then 0xffff else 0));
    if $-1 then ignore block $2 $0; ignore wait; .horiz ($2 + $-1) $1 $0
           else 0

/* vert x y0 y1  */
vert(3) =
    (if $1 < $0 then 1 else (if $0 < $1 then 0xffff else 0));
    if $-1 then ignore block $2 $1; ignore wait; .vert $2 ($1 + $-1) $0
           else 0

/* skewed_l x y  */
skewed_l(2) =
    if $1 & 1 then ignore memor_shr_skip (0xa000 + 0x80 * $0 + ($1 >> 1)) 0x10 0x08 0xff00 8;
                   memor_shr_skip (0xa001 + 0x80 * $0 + ($1 >> 1)) 0x10 0x08 0xff 0
              else memor_shr_skip (0xa000 + 0x80 * $0 + ($1 >> 1)) 0x10 0x08 0xff00 0

/* skewed_r x y  */
skewed_r(2) =
    if $1 & 1 then ignore memor_shl_skip (0xa000 + 0x80 * $0 + ($1 >> 1)) 0x10 0x08 0xff00 0;
                   memor_shl_skip (0xa001 + 0x80 * $0 + ($1 >> 1)) 0x10 0x08 0xff 8
              else memor_shl_skip (0xa000 + 0x80 * $0 + ($1 >> 1)) 0x10 0x08 0x00ff 0
'''

Now lets add our function and main function

In [562]:
prog +='''
/* square x y len  */
square(3) =
    ignore horiz $0 ($0 + $2) $1;  /* top line */
    ignore vert ($0 + $2) $1 ($1 + $2);  /* right line */
    ignore horiz ($0 + $2) $0 ($1 + $2);  /* bottom line */
    ignore vert $0 ($1 + $2) $1;  /* left line */
    0

set_pixel(2) =
    if $1 < $2 then ignore mem_poke 0xffff $1; set_pixel ($1 + 1) $2
    else 0

main(0) =
    square 0xa000 0xa000 100  /* top line */
'''


Now lets compile and print the code

In [563]:
cb.funcs(p(prog))
cb.code

['main',
 ('PUSH', 11),
 ('PUSH', 20),
 ('PUSH', 'main:0'),
 ('DUP', 2),
 ('DUP', 2),
 ('PUSH', 1),
 ('POP', 2),
 ('ALU', 'ADD'),
 ('DUP', 4),
 ('JMP', 'horiz'),
 'main:0',
 ('POP', 1),
 ('PUSH', 'main:1'),
 ('DUP', 1),
 ('DUP', 3),
 ('PUSH', 1),
 ('POP', 2),
 ('ALU', 'ADD'),
 ('DUP', 3),
 ('PUSH', 1),
 ('POP', 2),
 ('ALU', 'ADD'),
 ('JMP', 'vert'),
 'main:1',
 ('POP', 1),
 ('PUSH', 'main:2'),
 ('DUP', 1),
 ('PUSH', 1),
 ('POP', 2),
 ('ALU', 'SUB'),
 ('DUP', 3),
 ('DUP', 3),
 ('JMP', 'horiz'),
 'main:2',
 ('POP', 1),
 ('PUSH', 'main:3'),
 ('DUP', 2),
 ('DUP', 2),
 ('DUP', 4),
 ('JMP', 'vert'),
 'main:3',
 ('POP', 1),
 ('PUSH', 0),
 ('YANK', (1, 2)),
 ('POP', 2),
 ('RET', 1),
 'memset_skip',
 ('PUSH', 0),
 ('DUP', 2),
 ('POP', 2),
 ('ALU', 'LT'),
 ('POP', 1),
 ('JZ', 'memset_skip:0'),
 ('PUSH', 'memset_skip:1'),
 ('DUP', 1),
 ('DUP', 5),
 ('JMP', 'mem_poke'),
 'memset_skip:1',
 ('DUP', 4),
 ('DUP', 4),
 ('POP', 2),
 ('ALU', 'ADD'),
 ('DUP', 4),
 ('DUP', 4),
 ('PUSH', 1),
 ('POP', 2),
 (

As you can see, we get a list of tuples and labels (but we want a runnable code) so let's define the following function that will help us transform the list into a runnable code

In [564]:
def format_assembly(instruction_list):
    output = []
    for element in instruction_list:
        if isinstance(element, str):
            # Transform and append label
            transformed = element.replace(':', '_')
            output.append(transformed + ':')
        elif isinstance(element, tuple):
            inst = element[0]
            arg = element[1]
            if isinstance(arg, int):
                output.append(f"{inst} {arg}")
            elif isinstance(arg, str):
                transformed_arg = arg.replace(':', '_')
                output.append(f"{inst} {transformed_arg}")
            elif isinstance(arg, tuple):
                args = ' '.join(map(str, arg))
                output.append(f"{inst} {args}")
            else:
                raise ValueError("Unexpected argument type")
        else:
            raise ValueError("Unexpected element type")
    return '\n'.join(output)

Now let's create the code

In [565]:
code = format_assembly(cb.code)
print(code)

main:
PUSH 11
PUSH 20
PUSH main_0
DUP 2
DUP 2
PUSH 1
POP 2
ALU ADD
DUP 4
JMP horiz
main_0:
POP 1
PUSH main_1
DUP 1
DUP 3
PUSH 1
POP 2
ALU ADD
DUP 3
PUSH 1
POP 2
ALU ADD
JMP vert
main_1:
POP 1
PUSH main_2
DUP 1
PUSH 1
POP 2
ALU SUB
DUP 3
DUP 3
JMP horiz
main_2:
POP 1
PUSH main_3
DUP 2
DUP 2
DUP 4
JMP vert
main_3:
POP 1
PUSH 0
YANK 1 2
POP 2
RET 1
memset_skip:
PUSH 0
DUP 2
POP 2
ALU LT
POP 1
JZ memset_skip_0
PUSH memset_skip_1
DUP 1
DUP 5
JMP mem_poke
memset_skip_1:
DUP 4
DUP 4
POP 2
ALU ADD
DUP 4
DUP 4
PUSH 1
POP 2
ALU SUB
DUP 4
YANK 4 5
JMP memset_skip
JMP memset_skip_2
memset_skip_0:
PUSH 0
memset_skip_2:
YANK 1 4
POP 2
RET 1
memor_skip:
PUSH 0
DUP 2
POP 2
ALU LT
POP 1
JZ memor_skip_0
PUSH memor_skip_1
DUP 1
PUSH memor_skip_2
DUP 6
JMP mem_peek
memor_skip_2:
POP 2
ALU OR
DUP 5
JMP mem_poke
memor_skip_1:
DUP 4
DUP 4
POP 2
ALU ADD
DUP 4
DUP 4
PUSH 1
POP 2
ALU SUB
DUP 4
YANK 4 5
JMP memor_skip
JMP memor_skip_3
memor_skip_0:
PUSH 0
memor_skip_3:
YANK 1 4
POP 2
RET 1
wait:
PUSH wait_0
PUSH

After we have the code, let's verify that all 4 corners of the rectangle are white

In [566]:
# define a function that draws a square in IR code
stack = Array("stack", IntSort(), IntSort())
memory = Array("memory", IntSort(), IntSort())
sp, r0, r1 = Ints('sp r0 r1')


chcs = create_chcs(pre_condition=stack[0] == 0,
                   input_vars=[],
                   program=cb.code,
                   post_condition=And(
                                      memory[0] == 2,
                                      r1 == 2))

ValueError: Invalid program item: main